In [90]:
import numpy as np

## Load the data

In [156]:
subset = np.logical_not(np.load("data/missing_file_names_mask.npy"))
sample_hardness = np.load('data/resnet50_losses_final_weights.npy')[subset]
mouse_records  = np.load('data/mouse_record_interpolated.npy')

estimate_times=np.load('data/sample_estimate_times.npy')[subset]
worker_ids=np.load('data/sample_worker_ids.npy')[subset]
clean_subset = estimate_times < 3000
clean_subset = np.logical_and(clean_subset, estimate_times > 0)

sample_hardness = sample_hardness[clean_subset]
mouse_records = mouse_records[clean_subset]
estimate_times = estimate_times[clean_subset]
worker_ids = worker_ids[clean_subset]

In [125]:
not_zero = []
for i, el in enumerate(mouse_records):
    # print(el.shape, np.allclose(el, np.zeros_like(el)), el)
    if not np.allclose(el, np.zeros_like(el)):
        not_zero.append(i)
len(not_zero), len(mouse_records)

809947

In [157]:
mouse_records = mouse_records[not_zero]
sample_hardness = sample_hardness[not_zero]
estimate_times  = estimate_times[not_zero]
worker_ids  = worker_ids[not_zero]
worker_ids_int = np.zeros_like(worker_ids, dtype = np.float32)

In [171]:
for i, w in enumerate(np.unique(worker_ids)):
    worker_ids_int[np.where(worker_ids == w)] = i


In [172]:
worker_ids_int

array([619., 842., 923., ..., 213., 188., 231.], dtype=float32)

## Compute some pairings
The stored data will be (mouse_record_0, mouse_record_1, target, sample_hardness_0, sample_hardness_1, estimate_time_0, estimate_time_1, worker_id_0, worker_id_1, sample_1_position_in_the_grid, sample_1_position_in_the_grid)
sample_1_position_in_the_grid:{0:left,1:center,2:right}

In [177]:
NUM_SAMPLES = 2000000

In [182]:
idx0 = np.random.choice(len(sample_hardness), NUM_SAMPLES)
idx1 = np.random.choice(len(sample_hardness), NUM_SAMPLES)
invalid = idx0 == idx1
idx0 = idx0[~invalid]
idx1 = idx1[~invalid]

In [192]:
# sh0 + margin < sh1 -> target = 1
# sh1 + margin < sh0 -> target = 0
margin = 1
lower0 = sample_hardness[idx0] < sample_hardness[idx1]
over_margin_0 = sample_hardness[idx0] + margin < sample_hardness[idx1]
discard0 = np.logical_and(lower0, ~over_margin_0)
lower1 = sample_hardness[idx1] < sample_hardness[idx0]
over_margin_1 = sample_hardness[idx1] + margin < sample_hardness[idx0]
discard1 = np.logical_and(lower1, ~over_margin_1)
to_discard = np.logical_or(discard0, discard1)

idx0 = idx0[~to_discard]
idx1 = idx1[~to_discard]

In [191]:
data = np.zeros((len(idx0), mouse_records.shape[1]*mouse_records.shape[2]+mouse_records.shape[1]*mouse_records.shape[2]+7))
print(data.shape)
target = (sample_hardness[idx0]<sample_hardness[idx1])*1.

data[:, :30]=mouse_records[idx0][:, :, 0] # x-axis
data[:, 30:60]=mouse_records[idx0][:, :, 1] # y-axis
data[:, 60:90]=mouse_records[idx1][:, :, 0] # x-axis
data[:, 90:120]=mouse_records[idx1][:, :, 1] # x-axis
data[:, 120]=target
data[:, 121]=sample_hardness[idx0] # hardness
data[:, 122]=sample_hardness[idx1] # hardness
data[:, 123]=estimate_times[idx0] # estimate time
data[:, 124]=estimate_times[idx1] # estimate time
data[:, 125]=worker_ids_int[idx0] # worker id
data[:, 126]=worker_ids_int[idx1] # worker id
# data[:, 127]=position[idx0] # position
# data[:, 128]=position[idx1] # position


(881686, 127)


In [194]:
np.save('data/siamese_data_cleaned.npy', data)